In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

C:\Users\INBOTICS\.conda\envs\project\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\INBOTICS\.conda\envs\project\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\INBOTICS\.conda\envs\project\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


1 Physical GPUs, 1 Logical GPUs


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os,cv2
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential ,Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense ,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tqdm.notebook import tqdm_notebook
import time

In [3]:
# directory="Dataset/"

directory="bone_marrow_cell_dataset/"
classses=os.listdir(directory)
accuracy_list = []

In [4]:
classses

['ART',
 'BLA',
 'EBO',
 'EOS',
 'LYT',
 'MMZ',
 'MON',
 'MYB',
 'NGB',
 'NGS',
 'NIF',
 'PEB',
 'PLM',
 'PMO']

In [5]:
import os
import cv2
image_size=224
data1=[]#empty list
for category in classses:
    path = os.path.join(directory, category)
    print(path)
    label = classses.index(category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        arr = cv2.imread(img_path)#Reading Images
        arr = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
        new_arr = cv2.resize(arr, (image_size, image_size))#Resizing into same shape
        data1.append([new_arr, label])

bone_marrow_cell_dataset/ART
bone_marrow_cell_dataset/BLA
bone_marrow_cell_dataset/EBO
bone_marrow_cell_dataset/EOS
bone_marrow_cell_dataset/LYT
bone_marrow_cell_dataset/MMZ
bone_marrow_cell_dataset/MON
bone_marrow_cell_dataset/MYB
bone_marrow_cell_dataset/NGB
bone_marrow_cell_dataset/NGS
bone_marrow_cell_dataset/NIF
bone_marrow_cell_dataset/PEB
bone_marrow_cell_dataset/PLM
bone_marrow_cell_dataset/PMO


In [6]:
import random
random.shuffle(data1)

In [7]:
classses

['ART',
 'BLA',
 'EBO',
 'EOS',
 'LYT',
 'MMZ',
 'MON',
 'MYB',
 'NGB',
 'NGS',
 'NIF',
 'PEB',
 'PLM',
 'PMO']

In [8]:
x=[]#ndependent
y=[]#dependent
for features,labels in data1:
    x.append(features)
    y.append(labels)

In [9]:
x=np.array(x)
y=np.array(y)
#x=x/255 #feature scaling

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

# EfficientNetB7

In [11]:
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


base_model = applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224,224,3))

model_eff = Sequential()
#data_augmentation = keras.Sequential([layers.experimental.preprocessing.RandomRotation(0.1)])
model_eff.add(base_model)
model_eff.add(Flatten(input_shape=base_model.output_shape[1:]))
model_eff.add(Dense(512, activation='relu'))
model_eff.add(Dense(256, activation='relu'))
model_eff.add(Dense(128, activation='relu'))
model_eff.add(Dense(64, activation='relu'))
model_eff.add(Dense(len(classses), activation='softmax'))

model_eff = Model(inputs=model_eff.input, outputs=model_eff.output)
model_eff.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])


In [12]:
EfficientNet_progess_1 = model_eff.fit(x=X_train,y=y_train,epochs=10,validation_data= (X_test, y_test),batch_size=8)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[192,32,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_1/efficientnetb7/block2a_expand_conv/Conv2D (defined at \AppData\Local\Temp\ipykernel_15868\4281768224.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_71550]

Function call stack:
train_function


In [ ]:
model_eff.save("model/EfficientNetB7.hp5")


In [ ]:
plt.plot(EfficientNet_progess_1.history['accuracy'])
plt.plot(EfficientNet_progess_1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(EfficientNet_progess_1.history['loss'])
plt.plot(EfficientNet_progess_1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
Y_pred = model_eff.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

import seaborn as sns
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(cf_matrix, annot=True)

In [ ]:
import os
for i in os.listdir("testing/"):
    print(i)
    
    from tensorflow import keras
    model = keras.models.load_model('model/EfficientNetB7.hp5')
    from keras.preprocessing import image
    image_size=224
    #img = cv2.imread(path1+"//"+i)
    path="testing/"+i
    img = image.load_img(path, target_size=(image_size, image_size))
    x = image.img_to_array(img)
    print(type(x))
    img_4d=x.reshape(1,224,224,3)
    #img_4d=img_4d/255
    model = keras.models.load_model('model/VGGSKin.hp5')
    predictions = model.predict(img_4d)
    print(predictions[0])
    new_pred=np.argmax(predictions[0])
    print(new_pred)
#     ['COVID19', 'NORMAL', 'OTHERS', 'PNEUMONIA', 'TUBERCOLOSIS']
    dict1={0:'ART',1:'BLA',2:'EBO',3:'EOS',4:'LYT',5:'MMZ',6:'MON',7:'MYB',8:'NGB',9:'NGS',10:'NIF',11:'PEB',12:'PLM',13:'PMO'}
    list1=['ART','BLA','EBO','EOS','LYT','MMZ','MON','MYB','NGB','NGS','NIF','PEB','PLM','PMO']
    a=dict1[new_pred]
    
    print(str(a)+" Detected in Image")
    print("-----------------------------------------------")
    #break

